In [102]:
#correccion de precios dataset

In [103]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import train_test_split
import re
import statsmodels.api as sm
from sklearn import metrics
%matplotlib inline

In [104]:
#cargo el dataset
df = pd.read_csv('properatti.csv')

In [105]:
df.head()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...


In [106]:
#borrado de datos duplicados
df.drop_duplicates(inplace=True)

In [107]:
media = df['surface_covered_in_m2'].mean()
desvio = df['surface_covered_in_m2'].std()
sup_cubierta_out = len(df.loc[df['surface_covered_in_m2']- media >= (3*desvio),['surface_covered_in_m2']])
sup_cubierta_out

163

In [108]:
media = df['surface_total_in_m2'].mean()
desvio = df['surface_total_in_m2'].std()
sup_total_out = len(df.loc[df['surface_total_in_m2']- media >= (3*desvio),['surface_total_in_m2']])
sup_total_out

167

In [109]:
conditions = [(df['surface_total_in_m2'] < df['surface_covered_in_m2'] ) ,(df['surface_total_in_m2'] > df['surface_covered_in_m2'] )]

choices = [df['surface_covered_in_m2'],df['surface_total_in_m2'] ]

df['surface_total_in_m2'] = np.select(conditions, choices, df['surface_total_in_m2'])

In [110]:
#PRECIOS

In [111]:
evalua = [(df['price_usd_per_m2'] > 10000 ) & (df['price_usd_per_m2'] > df['price_per_m2'] )]
op = [df['price_aprox_usd'] /df['surface_covered_in_m2'] ]
df['price_usd_per_m2'] = np.select(evalua, op, df['price_usd_per_m2'])

In [112]:
evalua = [(df['place_name'] =='Villa Ballester') & (df['price_usd_per_m2'] > 10000)]   
op = [(df['price_aprox_usd']/17.831) ]
df['price_aprox_usd'] = np.select(evalua, op, df['price_aprox_usd'])

op = [(df['price_aprox_usd']/df['surface_covered_in_m2']) ]
df['price_usd_per_m2'] = np.select(evalua, op, df['price_per_m2'])

op = [(df['price_aprox_usd']/df['surface_total_in_m2']) ]
df['price_usd_per_m2'] = np.select(evalua, op, df['price_usd_per_m2'])

In [113]:
evalua = [(df['place_name'] =='Villa Ballester') & (df['price_usd_per_m2'] > 10000) &(df['currency'] =='USD')]   
op = [(df['price_aprox_usd']) ]
df['price'] = np.select(evalua, op, df['price'])

In [114]:
# Completo precio por metro nulos por precio metros cubiertos
df.precio_USD_per_m2_total=df.precio_USD_per_m2_total.fillna(value=df.precio_per_m2_cubierto)

AttributeError: 'DataFrame' object has no attribute 'precio_USD_per_m2_total'

In [115]:
df_precio = df.loc[:,['price','currency','price_aprox_local_currency','price_aprox_usd',\
                      'surface_total_in_m2','surface_covered_in_m2','price_usd_per_m2',\
                      'price_per_m2','description','title']]

In [116]:
#Manejo de outlier
Prec=df[ (df['price_usd_per_m2'] < 10000)]
PrProm=pd.DataFrame(Prec.groupby(['place_name', 'property_type'])['price_usd_per_m2'].mean())
PrProm.rename(columns={'price_usd_per_m2': 'PromPrecio'},inplace=True)
df=pd.merge(left=df,right=PrProm, how='left',left_on=['place_name','property_type'], right_on = ['place_name','property_type'])

In [117]:
#ajusto precios y relacionados dependiendo del promedio por barrio y tipo propiedad
evalua = [(df['price_usd_per_m2'] > 9999)]
op = [(df['PromPrecio']*df['surface_total_in_m2']) ]
df['price_aprox_usd'] = np.select(conditions, choices, df['price_aprox_usd'])
op = [(df['PromPrecio']) ]
df['price_usd_per_m2'] = np.select(conditions, choices, df['price_usd_per_m2'])
op = [(df['price_aprox_usd']/df['surface_covered_in_m2']) ]
df['price_per_m2'] = np.select(conditions, choices, df['price_per_m2'])

In [118]:
#ajuste de precios en boedo y san cristobal
evalua = [(df['place_name'] =='Boedo') & (df['price_usd_per_m2'] > df['PromPrecio']), (df['place_name'] =='San Cristobal') & (df['price_usd_per_m2'] > df['PromPrecio'])]
op = [(df['PromPrecio']*df['surface_total_in_m2']) ,(df['PromPrecio']*df['surface_total_in_m2'])]
df['price_aprox_usd'] = np.select(evalua, op, df['price_aprox_usd'])
op = [(df['PromPrecio']), (df['PromPrecio'])]
df['price_usd_per_m2'] = np.select(evalua, op, df['price_usd_per_m2'])
op = [(df['price_aprox_usd']/df['surface_covered_in_m2']), (df['price_aprox_usd']/df['surface_covered_in_m2'])]
df['price_per_m2'] = np.select(conditions, choices, df['price_per_m2'])

In [119]:
evalua = [(df['place_name'] =='Guillermo Hudson')]   
op = [2000]
df['price_per_m2'] = np.select(conditions, choices, df['price_per_m2'])

op =[( df['price_aprox_usd']/df['price_per_m2'])]
df['sup_cubierta_m2'] = np.select(evalua, op, df['surface_covered_in_m2'])
df['price_usd_per_m2'] = np.select(evalua, op, df['surface_total_in_m2'])

op = [2000]
df['price_usd_per_m2'] = np.select(evalua, op, df['price_usd_per_m2'])

In [120]:
evalua = [(df['place_name'] =='Los Pingüinos Country Club')]    #hay que corregir que no pise el campo poblado
Ping=pd.DataFrame(evalua)


pattern = r'(\d+\d+)'
regex = re.compile(pattern, flags= re.IGNORECASE)
df['Ping_title'] = df['title'].str.extract(regex, expand=True)


df['Ping']=df[['Ping_title']].apply(lambda x: (int(x['Ping_title'])*1000) if(np.all(pd.notnull(x['Ping_title']))) else x, axis = 1)


op =[( df['Ping'])]
df['price_aprox_usd'] = np.select(evalua, op, df['price_aprox_usd'])

choices =[( df['price_aprox_usd']/df['surface_total_in_m2'])]
df['price_usd_per_m2'] = np.select(evalua, op, df['price_usd_per_m2'])

In [121]:
#llenamos los precios con la * de PRECIOxM2 en USD y superficie_total_m2
precio_m2 = df.loc[(df['price_aprox_local_currency'].isnull())&(~df['price_aprox_usd'].isnull())&(~df['surface_total_in_m2'].isnull()) , ['price_aprox_usd']]
superf = df.loc[(df['price_aprox_local_currency'].isnull())&(~df['price_aprox_usd'].isnull())&(~df['surface_total_in_m2'].isnull()) , ['surface_total_in_m2']]
df.loc[df['price_aprox_local_currency'].isnull(), 'price_aprox_local_currency'] = precio_m2*superf

In [122]:
#llenamos precio por m2 en USD, con precio_USD_por_m2_total y superficie_total_m2
precio_m2 = df.loc[(df['price_aprox_usd'].isnull())&(~df['price_usd_per_m2'].isnull())&(~df['surface_total_in_m2'].isnull()) , ['price_usd_per_m2']]
superf = df.loc[(df['price_aprox_usd'].isnull())&(~df['price_usd_per_m2'].isnull())&(~df['surface_total_in_m2'].isnull()) , ['surface_total_in_m2']]
df.loc[df['price_aprox_usd'].isnull(), 'price_aprox_usd'] = precio_m2*superf

In [123]:
precio = df.loc[~(df['price_aprox_usd'].isnull())&(df['price_usd_per_m2'].isnull())&(~df['surface_total_in_m2'].isnull()) , ['price_aprox_usd']]
superficie = df.loc[~(df['price_aprox_usd'].isnull())&(df['price_usd_per_m2'].isnull())&(~df['surface_total_in_m2'].isnull()) , ['price_usd_per_m2']]
df.loc[df['price_usd_per_m2'].isnull(), 'price_usd_per_m2'] = precio/superficie

In [124]:
#eliminamos valores nulos
check = ['surface_covered_in_m2','surface_total_in_m2', 'price_aprox_local_currency', 'price_aprox_usd', 'price_usd_per_m2','rooms']

df = df.dropna(axis =0 , how = 'any', subset = check)

In [125]:
#verificamos la cantidad de nulos que quedaron
df.isnull().sum()

Unnamed: 0                        0
operation                         0
property_type                     0
place_name                        4
place_with_parent_names           0
country_name                      0
state_name                        0
geonames_id                    3602
lat-lon                        2532
lat                            2532
lon                            2532
price                             0
currency                          0
price_aprox_local_currency        0
price_aprox_usd                   0
surface_total_in_m2               0
surface_covered_in_m2             0
price_usd_per_m2                  0
price_per_m2                      0
floor                         22263
rooms                             0
expenses                      21703
properati_url                     0
description                       0
title                             0
image_thumbnail                 108
PromPrecio                       93
sup_cubierta_m2             

In [126]:
df.price_per_m2.value_counts(sort=True)

50.000000       267
60.000000       245
45.000000       232
300.000000      213
80.000000       205
70.000000       204
55.000000       199
48.000000       191
52.000000       189
35.000000       182
200.000000      177
43.000000       176
40.000000       175
58.000000       174
62.000000       160
54.000000       159
65.000000       158
68.000000       157
46.000000       157
75.000000       152
53.000000       152
100.000000      152
110.000000      150
90.000000       148
42.000000       148
38.000000       146
36.000000       141
56.000000       139
120.000000      139
2000.000000     133
               ... 
1912.500000       1
760.000000        1
5428.571429       1
33209.302326      1
2791.666667       1
5840.000000       1
27666.666667      1
25967.090909      1
3050.847458       1
27641.509434      1
50.279330         1
2575.000000       1
2256.410256       1
1598.333333       1
715.409091        1
2434.210526       1
6025.714286       1
1985.815603       1
1717.948718       1


In [127]:
##creacion de df para la corrida del modelo
df_modelo = df.loc[:,['price','currency','price_aprox_local_currency','price_aprox_usd',\
                      'surface_total_in_m2','surface_covered_in_m2','price_usd_per_m2',\
                      'price_per_m2','floor','property_type','description','title','rooms','Localidad','state_name']]  ## Parte con Precios y Superficies
                     

C:\Users\guido\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [128]:
df_modelo

,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,property_type,description,title,rooms,Localidad,state_name
13,111700.0,USD,1970890.65,50.00,50.0,30.0,50.0,50.000000,NaN,apartment,Torre I Mondrian. 3 ambientes con terraza y d...,Vitraux Palermo,1.0,NaN,Capital Federal
14,147900.0,USD,2609621.55,42.00,42.0,31.0,42.0,42.000000,NaN,apartment,Torre II Dalí. Ambiente unico divisible.Vitrau...,Vitraux Palermo,1.0,NaN,Capital Federal
16,239000.0,USD,4217035.50,140.00,140.0,98.0,140.0,140.000000,NaN,PH,Ventas Mataderos al frente Duplex 4 amb.- Plan...,VENTA-MATADEROS-DUPLEX 4 AMB.,4.0,NaN,Capital Federal
17,315000.0,USD,5558017.50,176.00,176.0,78.0,176.0,176.000000,NaN,apartment,Excelente 3 ambientes en un 6 piso y con terra...,3 ambientes en venta en Acqua Golf 1 en Nordel...,3.0,NaN,Bs.As. G.B.A. Zona Norte
19,350000.0,USD,6175575.00,104.00,104.0,96.0,104.0,104.000000,NaN,apartment,Excelente semipiso al contra frente en Bulnes ...,"Bulnes y Libertador: espectacular pulmón, con ...",3.0,NaN,Capital Federal
21,270500.0,USD,4772837.25,118.00,118.0,73.0,118.0,118.000000,NaN,apartment,"EXCELENTE ZONA, MULTIPLES MEDIOS DE TRANSPORTE...",Departamento de 4 ambientes en Venta en Palermo,4.0,NaN,Capital Federal
23,82000.0,USD,1446849.00,48.00,48.0,42.0,48.0,48.000000,NaN,apartment,"Hermoso departamento, ubicado en la calle Almi...","Excelente departamento, Impecable!!! APTO CRED...",2.0,NaN,Bs.As. G.B.A. Zona Norte
24,355000.0,USD,6263797.50,355000.00,160.0,160.0,160.0,2218.750000,NaN,apartment,EXCLUSIVO DEPARTAMENTO DE EXCELENTE CATEGORIA...,VENTA DE POZO- DEPTO 4 AMBIENTES- CASTELAR.,4.0,NaN,Bs.As. G.B.A. Zona Oeste
25,358550.0,USD,6326435.47,358550.00,160.0,160.0,160.0,2240.937500,NaN,apartment,EXCLUSIVO DEPARTAMENTO DE EXCELENTE CATEGORÍA...,VENTA DE POZO- DEPTO 4 AMBIENTES- CASTELAR.,4.0,NaN,Bs.As. G.B.A. Zona Oeste
28,362150.0,USD,6389955.67,362150.00,160.0,160.0,160.0,2263.437500,NaN,apartment,EXCLUSIVO DEPARTAMENTO DE EXCELENTE CATEGORÍA...,VENTA DE POZO- DEPTO 4 AMBIENTES- CASTELAR.,4.0,NaN,Bs.As. G.B.A. Zona Oeste


In [129]:
df_posicion_geografica = df.loc[:,['place_name','state_name','country_name','place_with_parent_names' \
                                   ,'geonames_id','lat','lon','lat-lon', \
                                   'describe','title']] ### Parte con Posición Geográfica
#localidad
df_localidad_ex = pd.DataFrame(item for item in df.place_with_parent_names.str.split('|'))
df_localidad_ex[df_localidad_ex[4] =='coordenadas 34.255511'] 
df_localidad_ex.iloc[7367,4] ='Nordelta'

df_posicion_geografica['pais_ex'] = df_localidad_ex[1]
df_posicion_geografica['provincia_ex'] = df_localidad_ex[2]
df_posicion_geografica['localidad_ex'] = df_localidad_ex[3]
df_posicion_geografica['barrio_ex'] = df_localidad_ex[4]
df_posicion_geografica.place_name.fillna(df_posicion_geografica.localidad_ex,inplace=True)
df_posicion_geografica.barrio_ex =df_posicion_geografica.apply(lambda x: x.localidad_ex 
                                       if x.barrio_ex == '' or x.barrio_ex ==None 
                                       else x.barrio_ex,1)


In [130]:
df_modelo.columns

Index(['price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd',
       'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2',
       'price_per_m2', 'floor', 'property_type', 'description', 'title',
       'rooms', 'Localidad', 'state_name'],
      dtype='object')

In [131]:
df_modelo['state_name'] = df_posicion_geografica['provincia_ex']
df_modelo['Localidad'] = df_posicion_geografica['localidad_ex']
df_modelo['place_name'] = df_posicion_geografica['barrio_ex']

In [132]:
df_modelo

,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,property_type,description,title,rooms,Localidad,state_name,place_name
13,111700.0,USD,1970890.65,50.00,50.0,30.0,50.0,50.000000,NaN,apartment,Torre I Mondrian. 3 ambientes con terraza y d...,Vitraux Palermo,1.0,Vicente López,Bs.As. G.B.A. Zona Norte,Munro
14,147900.0,USD,2609621.55,42.00,42.0,31.0,42.0,42.000000,NaN,apartment,Torre II Dalí. Ambiente unico divisible.Vitrau...,Vitraux Palermo,1.0,Boedo,Capital Federal,Boedo
16,239000.0,USD,4217035.50,140.00,140.0,98.0,140.0,140.000000,NaN,PH,Ventas Mataderos al frente Duplex 4 amb.- Plan...,VENTA-MATADEROS-DUPLEX 4 AMB.,4.0,Pilar,Bs.As. G.B.A. Zona Norte,Del Viso
17,315000.0,USD,5558017.50,176.00,176.0,78.0,176.0,176.000000,NaN,apartment,Excelente 3 ambientes en un 6 piso y con terra...,3 ambientes en venta en Acqua Golf 1 en Nordel...,3.0,Pilar,Bs.As. G.B.A. Zona Norte,Pilar
19,350000.0,USD,6175575.00,104.00,104.0,96.0,104.0,104.000000,NaN,apartment,Excelente semipiso al contra frente en Bulnes ...,"Bulnes y Libertador: espectacular pulmón, con ...",3.0,Pilar,Bs.As. G.B.A. Zona Norte,Pilar
21,270500.0,USD,4772837.25,118.00,118.0,73.0,118.0,118.000000,NaN,apartment,"EXCELENTE ZONA, MULTIPLES MEDIOS DE TRANSPORTE...",Departamento de 4 ambientes en Venta en Palermo,4.0,San Isidro,Bs.As. G.B.A. Zona Norte,San Isidro
23,82000.0,USD,1446849.00,48.00,48.0,42.0,48.0,48.000000,NaN,apartment,"Hermoso departamento, ubicado en la calle Almi...","Excelente departamento, Impecable!!! APTO CRED...",2.0,Caballito,Capital Federal,Caballito
24,355000.0,USD,6263797.50,355000.00,160.0,160.0,160.0,2218.750000,NaN,apartment,EXCLUSIVO DEPARTAMENTO DE EXCELENTE CATEGORIA...,VENTA DE POZO- DEPTO 4 AMBIENTES- CASTELAR.,4.0,Caballito,Capital Federal,Caballito
25,358550.0,USD,6326435.47,358550.00,160.0,160.0,160.0,2240.937500,NaN,apartment,EXCLUSIVO DEPARTAMENTO DE EXCELENTE CATEGORÍA...,VENTA DE POZO- DEPTO 4 AMBIENTES- CASTELAR.,4.0,Caballito,Capital Federal,Caballito
28,362150.0,USD,6389955.67,362150.00,160.0,160.0,160.0,2263.437500,NaN,apartment,EXCLUSIVO DEPARTAMENTO DE EXCELENTE CATEGORÍA...,VENTA DE POZO- DEPTO 4 AMBIENTES- CASTELAR.,4.0,Mar del Plata,Buenos Aires Costa Atlántica,Mar del Plata


# Capeo de Variables

In [133]:
df_modelo=df_modelo.drop(columns=['description','title','currency','floor'])

In [134]:
df_modelo.describe()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms
count,2.364900e+04,2.364900e+04,2.364900e+04,23649.000000,23649.000000,23649.000000,23649.000000,23649.000000
mean,3.741026e+05,4.077881e+06,6.906821e+04,191.352192,93.647131,191.352192,2288.864121,2.845575
std,7.067307e+05,5.699671e+06,2.178585e+05,1954.841664,361.913409,1954.841664,6994.912207,1.463615
min,5.500000e+03,8.234017e+04,2.000000e+01,10.000000,1.000000,10.000000,20.000000,1.000000
25%,9.800000e+04,1.583309e+06,7.100000e+01,46.000000,42.000000,46.000000,71.000000,2.000000
50%,1.580000e+05,2.470230e+06,2.180000e+02,71.000000,63.000000,71.000000,217.000000,3.000000
75%,3.148090e+05,4.322902e+06,8.000000e+04,145.000000,105.000000,145.000000,1875.000000,4.000000
max,3.000000e+07,1.499782e+08,8.500000e+06,200000.000000,49000.000000,200000.000000,375000.000000,32.000000


In [135]:
df_modelo.head()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,property_type,rooms,Localidad,state_name,place_name
13,111700.0,1970890.65,50.0,50.0,30.0,50.0,50.0,apartment,1.0,Vicente López,Bs.As. G.B.A. Zona Norte,Munro
14,147900.0,2609621.55,42.0,42.0,31.0,42.0,42.0,apartment,1.0,Boedo,Capital Federal,Boedo
16,239000.0,4217035.50,140.0,140.0,98.0,140.0,140.0,PH,4.0,Pilar,Bs.As. G.B.A. Zona Norte,Del Viso
17,315000.0,5558017.50,176.0,176.0,78.0,176.0,176.0,apartment,3.0,Pilar,Bs.As. G.B.A. Zona Norte,Pilar
19,350000.0,6175575.00,104.0,104.0,96.0,104.0,104.0,apartment,3.0,Pilar,Bs.As. G.B.A. Zona Norte,Pilar


In [136]:
df_modelo.price.max(), df_modelo.price_usd_per_m2.max(), df_modelo.surface_total_in_m2.max(), df_modelo.surface_covered_in_m2.max()

(30000000.0, 200000.0, 200000.0, 49000.0)

In [137]:
df_modelo.price.quantile(0.99), df_modelo.price_usd_per_m2.quantile(0.99), df_modelo.surface_total_in_m2.quantile(0.99), df_modelo.surface_covered_in_m2.quantile(0.99)

(3500000.0, 1200.0, 1200.0, 392.08000000000175)

# Segmentación

In [138]:
df_capital_federal = df_modelo[df_modelo.state_name == 'Capital Federal']
df_capital_federal.head()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,property_type,rooms,Localidad,state_name,place_name
14,147900.0,2609621.55,42.0,42.0,31.0,42.0,42.0000,apartment,1.0,Boedo,Capital Federal,Boedo
23,82000.0,1446849.00,48.0,48.0,42.0,48.0,48.0000,apartment,2.0,Caballito,Capital Federal,Caballito
24,355000.0,6263797.50,355000.0,160.0,160.0,160.0,2218.7500,apartment,4.0,Caballito,Capital Federal,Caballito
25,358550.0,6326435.47,358550.0,160.0,160.0,160.0,2240.9375,apartment,4.0,Caballito,Capital Federal,Caballito
38,480000.0,8469360.00,220.0,220.0,130.0,220.0,220.0000,apartment,3.0,Almagro,Capital Federal,Almagro


In [139]:
df_zona_norte = df_modelo[df_modelo.state_name == 'Bs.As. G.B.A. Zona Norte']
df_zona_norte.head()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,property_type,rooms,Localidad,state_name,place_name
13,111700.0,1970890.65,50.0,50.0,30.0,50.0,50.0,apartment,1.0,Vicente López,Bs.As. G.B.A. Zona Norte,Munro
16,239000.0,4217035.50,140.0,140.0,98.0,140.0,140.0,PH,4.0,Pilar,Bs.As. G.B.A. Zona Norte,Del Viso
17,315000.0,5558017.50,176.0,176.0,78.0,176.0,176.0,apartment,3.0,Pilar,Bs.As. G.B.A. Zona Norte,Pilar
19,350000.0,6175575.00,104.0,104.0,96.0,104.0,104.0,apartment,3.0,Pilar,Bs.As. G.B.A. Zona Norte,Pilar
21,270500.0,4772837.25,118.0,118.0,73.0,118.0,118.0,apartment,4.0,San Isidro,Bs.As. G.B.A. Zona Norte,San Isidro


In [140]:
df_zona_sur = df_modelo[df_modelo.state_name == 'Bs.As. G.B.A. Zona Sur']
df_zona_sur.head()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,property_type,rooms,Localidad,state_name,place_name
209,235000.0,4146457.5,68.0,68.0,48.0,68.0,68.0,apartment,2.0,Quilmes,Bs.As. G.B.A. Zona Sur,Quilmes
226,135000.0,2382007.5,45.0,45.0,39.0,45.0,45.0,apartment,1.0,La Plata,Bs.As. G.B.A. Zona Sur,La Plata
260,305000.0,5381572.5,120.0,120.0,114.0,120.0,120.0,apartment,4.0,Cañuelas,Bs.As. G.B.A. Zona Sur,Cañuelas
280,120000.0,2117340.0,234.0,234.0,131.0,234.0,234.0,house,3.0,Almirante Brown,Bs.As. G.B.A. Zona Sur,Burzaco
292,140000.0,2470230.0,300.0,300.0,175.0,300.0,300.0,house,3.0,La Plata,Bs.As. G.B.A. Zona Sur,La Plata


In [141]:
df_zona_oeste = df_modelo[df_modelo.state_name == 'Bs.As. G.B.A. Zona Oeste']
df_zona_oeste.head()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,property_type,rooms,Localidad,state_name,place_name
54,220410.0,3889024.24,81.0,81.0,75.0,81.0,81.0,apartment,3.0,Tres de Febrero,Bs.As. G.B.A. Zona Oeste,Sáenz Peña
55,222780.0,3930841.71,81.0,81.0,75.0,81.0,81.0,apartment,3.0,Tres de Febrero,Bs.As. G.B.A. Zona Oeste,Sáenz Peña
216,550000.0,9704475.00,160.0,160.0,150.0,160.0,160.0,apartment,4.0,Tres de Febrero,Bs.As. G.B.A. Zona Oeste,Villa Raffo
218,62000.0,1093959.00,33.0,33.0,31.0,33.0,33.0,apartment,1.0,Tres de Febrero,Bs.As. G.B.A. Zona Oeste,Ciudad Jardín Lomas del Palomar
244,550000.0,9704475.00,109.0,109.0,99.0,109.0,109.0,apartment,4.0,Morón,Bs.As. G.B.A. Zona Oeste,Haedo


In [142]:
df_costa_atlantica = df_modelo[df_modelo.state_name == 'Buenos Aires Costa Atlántica']
df_costa_atlantica.head()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,property_type,rooms,Localidad,state_name,place_name
28,362150.0,6389955.67,362150.0,160.0,160.0,160.0,2263.4375,apartment,4.0,Mar del Plata,Buenos Aires Costa Atlántica,Mar del Plata
33,325000.0,5734462.50,186.0,186.0,83.0,186.0,186.0000,apartment,3.0,Mar del Plata,Buenos Aires Costa Atlántica,Mar del Plata
34,169000.0,2981920.50,172.0,172.0,160.0,172.0,172.0000,PH,6.0,Mar del Plata,Buenos Aires Costa Atlántica,Mar del Plata
35,145000.0,2558452.50,80.0,80.0,77.0,80.0,80.0000,apartment,4.0,Mar del Plata,Buenos Aires Costa Atlántica,Mar del Plata
43,79000.0,1393915.50,79000.0,32.0,32.0,32.0,2468.7500,apartment,1.0,Mar del Plata,Buenos Aires Costa Atlántica,Mar del Plata


In [143]:
df_cordoba = df_modelo[df_modelo.Localidad == 'Córdoba']
df_cordoba.head()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,property_type,rooms,Localidad,state_name,place_name
140,100000.0,1764450.0,112.0,112.0,63.0,112.0,112.0,apartment,3.0,Córdoba,Córdoba,Córdoba
150,85000.0,1499782.5,97.0,97.0,85.0,97.0,97.0,apartment,3.0,Córdoba,Córdoba,Córdoba
798,92000.0,1623294.0,92000.0,40.0,40.0,40.0,2300.0,apartment,2.0,Córdoba,Córdoba,Córdoba
800,88000.0,1552716.0,88000.0,40.0,40.0,40.0,2200.0,apartment,2.0,Córdoba,Córdoba,Córdoba
816,360000.0,6352020.0,147.0,147.0,143.0,147.0,147.0,apartment,4.0,Córdoba,Córdoba,Córdoba


In [144]:
df_rosario = df_modelo[df_modelo.Localidad == 'Rosario']
df_rosario.head()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,property_type,rooms,Localidad,state_name,place_name
291,95000.0,1676227.5,330.0,330.0,120.0,330.0,330.000000,house,3.0,Rosario,Santa Fe,Rosario
559,165000.0,2911342.5,165000.0,150.0,150.0,150.0,1100.000000,PH,3.0,Rosario,Santa Fe,Rosario
571,850000.0,14997825.0,850000.0,147.0,147.0,147.0,5782.312925,apartment,4.0,Rosario,Santa Fe,Rosario
573,275000.0,4852237.5,275000.0,91.0,91.0,91.0,3021.978022,apartment,3.0,Rosario,Santa Fe,Rosario
574,230000.0,4058235.0,230000.0,74.0,74.0,74.0,3108.108108,apartment,3.0,Rosario,Santa Fe,Rosario


In [145]:
df_capital_federal.corr()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms
price,1.000000,0.409900,0.186486,0.015205,0.209711,0.015205,0.183022,0.180301
price_aprox_local_currency,0.409900,1.000000,0.508901,0.012512,0.145977,0.012512,-0.002679,0.403312
price_aprox_usd,0.186486,0.508901,1.000000,0.010546,0.050708,0.010546,0.174259,0.116737
surface_total_in_m2,0.015205,0.012512,0.010546,1.000000,0.098905,1.000000,0.695314,0.040806
surface_covered_in_m2,0.209711,0.145977,0.050708,0.098905,1.000000,0.098905,0.050334,0.182896
price_usd_per_m2,0.015205,0.012512,0.010546,1.000000,0.098905,1.000000,0.695314,0.040806
price_per_m2,0.183022,-0.002679,0.174259,0.695314,0.050334,0.695314,1.000000,-0.042608
rooms,0.180301,0.403312,0.116737,0.040806,0.182896,0.040806,-0.042608,1.000000


In [146]:
df_zona_norte.corr()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms
price,1.000000,0.319053,0.188810,0.050794,0.129414,0.050794,0.271464,0.123372
price_aprox_local_currency,0.319053,1.000000,0.707518,0.224619,0.455556,0.224619,0.008719,0.402391
price_aprox_usd,0.188810,0.707518,1.000000,0.031970,0.273788,0.031970,0.183646,0.145407
surface_total_in_m2,0.050794,0.224619,0.031970,1.000000,0.535455,1.000000,0.006616,0.446152
surface_covered_in_m2,0.129414,0.455556,0.273788,0.535455,1.000000,0.535455,-0.016983,0.579518
price_usd_per_m2,0.050794,0.224619,0.031970,1.000000,0.535455,1.000000,0.006616,0.446152
price_per_m2,0.271464,0.008719,0.183646,0.006616,-0.016983,0.006616,1.000000,-0.070885
rooms,0.123372,0.402391,0.145407,0.446152,0.579518,0.446152,-0.070885,1.000000


In [147]:
df_zona_sur.corr()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms
price,1.000000,0.258057,0.164626,0.040845,0.186023,0.040845,0.575807,0.156069
price_aprox_local_currency,0.258057,1.000000,0.508470,0.194104,0.633021,0.194104,0.013491,0.442306
price_aprox_usd,0.164626,0.508470,1.000000,-0.052653,0.277520,-0.052653,0.276214,0.208540
surface_total_in_m2,0.040845,0.194104,-0.052653,1.000000,0.406612,1.000000,0.068411,0.272908
surface_covered_in_m2,0.186023,0.633021,0.277520,0.406612,1.000000,0.406612,-0.015149,0.672314
price_usd_per_m2,0.040845,0.194104,-0.052653,1.000000,0.406612,1.000000,0.068411,0.272908
price_per_m2,0.575807,0.013491,0.276214,0.068411,-0.015149,0.068411,1.000000,0.027273
rooms,0.156069,0.442306,0.208540,0.272908,0.672314,0.272908,0.027273,1.000000


In [148]:
df_zona_oeste.corr()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms
price,1.000000,0.325510,0.186861,0.028892,0.178471,0.028892,0.247847,0.155149
price_aprox_local_currency,0.325510,1.000000,0.703269,0.160126,0.582301,0.160126,0.037676,0.422856
price_aprox_usd,0.186861,0.703269,1.000000,-0.007333,0.301223,-0.007333,0.221325,0.159301
surface_total_in_m2,0.028892,0.160126,-0.007333,1.000000,0.346852,1.000000,0.115131,0.362239
surface_covered_in_m2,0.178471,0.582301,0.301223,0.346852,1.000000,0.346852,0.005160,0.621249
price_usd_per_m2,0.028892,0.160126,-0.007333,1.000000,0.346852,1.000000,0.115131,0.362239
price_per_m2,0.247847,0.037676,0.221325,0.115131,0.005160,0.115131,1.000000,0.000798
rooms,0.155149,0.422856,0.159301,0.362239,0.621249,0.362239,0.000798,1.000000


In [149]:
df_costa_atlantica.corr()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms
price,1.000000,0.443849,0.158895,0.112446,0.285846,0.112446,0.361028,0.219305
price_aprox_local_currency,0.443849,1.000000,0.357041,0.282344,0.638397,0.282344,-0.054900,0.440490
price_aprox_usd,0.158895,0.357041,1.000000,-0.041249,0.180897,-0.041249,0.185329,0.097115
surface_total_in_m2,0.112446,0.282344,-0.041249,1.000000,0.636535,1.000000,-0.067480,0.517441
surface_covered_in_m2,0.285846,0.638397,0.180897,0.636535,1.000000,0.636535,-0.087498,0.736756
price_usd_per_m2,0.112446,0.282344,-0.041249,1.000000,0.636535,1.000000,-0.067480,0.517441
price_per_m2,0.361028,-0.054900,0.185329,-0.067480,-0.087498,-0.067480,1.000000,-0.112368
rooms,0.219305,0.440490,0.097115,0.517441,0.736756,0.517441,-0.112368,1.000000


In [150]:
df_cordoba.corr()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms
price,1.000000,0.469919,0.043270,0.213913,0.322569,0.213913,0.303584,0.302478
price_aprox_local_currency,0.469919,1.000000,0.140442,0.401365,0.708601,0.401365,-0.118568,0.481115
price_aprox_usd,0.043270,0.140442,1.000000,-0.041638,0.199241,-0.041638,0.240262,0.248569
surface_total_in_m2,0.213913,0.401365,-0.041638,1.000000,0.705821,1.000000,-0.111056,0.574838
surface_covered_in_m2,0.322569,0.708601,0.199241,0.705821,1.000000,0.705821,-0.107086,0.733513
price_usd_per_m2,0.213913,0.401365,-0.041638,1.000000,0.705821,1.000000,-0.111056,0.574838
price_per_m2,0.303584,-0.118568,0.240262,-0.111056,-0.107086,-0.111056,1.000000,-0.029796
rooms,0.302478,0.481115,0.248569,0.574838,0.733513,0.574838,-0.029796,1.000000


In [151]:
df_rosario.corr()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms
price,1.000000,0.274129,0.263315,0.034871,0.208997,0.034871,0.680721,0.196341
price_aprox_local_currency,0.274129,1.000000,0.621018,0.205653,0.672646,0.205653,0.009538,0.354830
price_aprox_usd,0.263315,0.621018,1.000000,0.015711,0.350850,0.015711,0.231439,0.103646
surface_total_in_m2,0.034871,0.205653,0.015711,1.000000,0.444399,1.000000,-0.041837,0.309231
surface_covered_in_m2,0.208997,0.672646,0.350850,0.444399,1.000000,0.444399,-0.028935,0.653309
price_usd_per_m2,0.034871,0.205653,0.015711,1.000000,0.444399,1.000000,-0.041837,0.309231
price_per_m2,0.680721,0.009538,0.231439,-0.041837,-0.028935,-0.041837,1.000000,-0.075271
rooms,0.196341,0.354830,0.103646,0.309231,0.653309,0.309231,-0.075271,1.000000


In [152]:
df_zona_norte.head()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,property_type,rooms,Localidad,state_name,place_name
13,111700.0,1970890.65,50.0,50.0,30.0,50.0,50.0,apartment,1.0,Vicente López,Bs.As. G.B.A. Zona Norte,Munro
16,239000.0,4217035.50,140.0,140.0,98.0,140.0,140.0,PH,4.0,Pilar,Bs.As. G.B.A. Zona Norte,Del Viso
17,315000.0,5558017.50,176.0,176.0,78.0,176.0,176.0,apartment,3.0,Pilar,Bs.As. G.B.A. Zona Norte,Pilar
19,350000.0,6175575.00,104.0,104.0,96.0,104.0,104.0,apartment,3.0,Pilar,Bs.As. G.B.A. Zona Norte,Pilar
21,270500.0,4772837.25,118.0,118.0,73.0,118.0,118.0,apartment,4.0,San Isidro,Bs.As. G.B.A. Zona Norte,San Isidro


In [153]:
df_capital_federal=df_capital_federal.drop(columns=['Localidad','state_name'])

In [154]:
df_zona_norte=df_zona_norte.drop(columns=['Localidad','state_name'])

In [155]:
df_zona_sur=df_zona_sur.drop(columns=['Localidad','state_name'])

In [156]:
df_zona_oeste=df_zona_oeste.drop(columns=['Localidad','state_name'])

In [157]:
df_costa_atlantica=df_costa_atlantica.drop(columns=['Localidad','state_name'])

In [158]:
df_cordoba=df_cordoba.drop(columns=['Localidad','state_name','place_name'])

In [159]:
df_rosario=df_rosario.drop(columns=['Localidad','state_name','place_name'])

In [160]:
#df_modelo.to_excel("modelo.xlsx")

In [161]:
df_dummies_capital=pd.get_dummies(df_capital_federal,columns=['place_name','property_type'], drop_first=True)

In [162]:
df_dummies_capital.describe()

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,rooms,place_name_Abasto,place_name_Agronomía,...,place_name_Villa Luro,place_name_Villa Ortuzar,place_name_Villa Pueyrredón,place_name_Villa Real,place_name_Villa Santa Rita,place_name_Villa Urquiza,place_name_Villa del Parque,property_type_apartment,property_type_house,property_type_store
count,2.750000e+03,2.750000e+03,2.750000e+03,2750.000000,2750.000000,2750.000000,2750.000000,2750.000000,2750.000000,2750.000000,...,2750.0000,2750.000000,2750.000000,2750.000000,2750.000000,2750.000000,2750.000000,2750.000000,2750.000000,2750.000000
mean,3.454794e+05,4.012177e+06,6.442896e+04,267.325091,105.772364,267.325091,1478.370327,2.996000,0.001818,0.002182,...,0.0080,0.001818,0.002182,0.001818,0.002545,0.032727,0.009455,0.733818,0.191273,0.003636
std,6.423798e+05,4.994345e+06,1.970886e+05,3859.505492,363.681425,3859.505492,5463.337434,1.483136,0.042609,0.046667,...,0.0891,0.042609,0.046667,0.042609,0.050397,0.177954,0.096791,0.442041,0.393375,0.060203
min,2.500000e+04,1.338030e+05,2.200000e+01,16.000000,6.000000,16.000000,22.000000,1.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.050000e+05,1.676228e+06,7.000000e+01,50.000000,45.000000,50.000000,70.000000,2.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.600000e+05,2.621476e+06,2.000000e+02,79.000000,68.000000,79.000000,200.000000,3.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,2.950000e+05,4.411125e+06,7.187500e+04,170.750000,121.000000,170.750000,1683.552632,4.000000,0.000000,0.000000,...,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1.088601e+07,7.057800e+07,4.000000e+06,200000.000000,18615.000000,200000.000000,200000.000000,14.000000,1.000000,1.000000,...,1.0000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [163]:
df_dummies_norte=pd.get_dummies(df_zona_norte,columns=['place_name','property_type'], drop_first=True)

In [164]:
df_dummies_sur=pd.get_dummies(df_zona_sur,columns=['place_name','property_type'], drop_first=True)

In [165]:
df_dummies_oeste=pd.get_dummies(df_zona_oeste,columns=['place_name','property_type'], drop_first=True)

In [166]:
df_dummies_costa=pd.get_dummies(df_costa_atlantica,columns=['place_name','property_type'], drop_first=True)

In [167]:
df_dummies_cordoba=pd.get_dummies(df_cordoba,columns=['property_type'], drop_first=True)

In [168]:
df_dummies_rosario=pd.get_dummies(df_rosario,columns=['property_type'], drop_first=True)

In [169]:
#df_dummies.columns

In [170]:
#modelo_capital
model =LinearRegression(fit_intercept = True)
modelRidge=Ridge()
modelLasso=Lasso()
y = df_dummies_capital['price_usd_per_m2']
X = df_dummies_capital.drop(columns=['price_usd_per_m2'])
Xtrain,Xtest,ytrain,ytest= train_test_split(X,y,\
                                     random_state=1)

In [171]:
Xtrain.shape,ytrain.shape

((2062, 68), (2062,))

In [172]:
Xtest.shape,ytest.shape

((688, 68), (688,))

In [173]:
model.fit(Xtrain,ytrain)
modelRidge.fit(Xtrain,ytrain)
modelLasso.fit(Xtrain,ytrain)

C:\Users\guido\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:147: LinAlgWarning: Ill-conditioned matrix (rcond=2.13942e-17): result may not be accurate.
  overwrite_a=True).T


Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [174]:
model.intercept_

1.8758328224066645e-12

In [175]:
df_coef=pd.DataFrame(model.coef_).T
df_coef.columns = df_dummies_capital.drop(columns=['price_usd_per_m2']).columns

In [176]:
df_coef

,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_per_m2,rooms,place_name_Abasto,place_name_Agronomía,place_name_Almagro,...,place_name_Villa Luro,place_name_Villa Ortuzar,place_name_Villa Pueyrredón,place_name_Villa Real,place_name_Villa Santa Rita,place_name_Villa Urquiza,place_name_Villa del Parque,property_type_apartment,property_type_house,property_type_store
0,2.890601e-17,-5.479287e-17,3.223116e-15,1.0,-3.219647e-15,-5.551115e-16,4.622934e-14,3.638203e-14,6.646267e-14,4.056125e-14,...,-9.102332e-15,1.162688e-13,2.059729e-13,-1.928116e-13,1.418519e-13,4.343963e-14,-5.188095e-14,-2.402568e-13,-1.659034e-13,-3.174159e-13


In [177]:
ypred=model.predict(Xtest)
ypredRidge=modelRidge.predict(Xtest)
ypredLasso=modelLasso.predict(Xtest)

In [178]:
print('Lineal: ',metrics.r2_score(ytest,ypred))
print('Ridge: ',metrics.r2_score(ytest,ypredRidge))
print('Lasso: ',metrics.r2_score(ytest,ypredLasso))


Lineal:  1.0
Ridge:  1.0
Lasso:  0.999999662694755
